# Experiments

In this notebook is implemented the three experiments described in the paper.

## Header

Libraries and additional programs

In [33]:
import matplotlib
import matplotlib.pyplot as plt
import numpy
import random
from scipy.stats.stats import pearsonr

%matplotlib inline

pathh = '../Data/' 
#exp_file = 'ExperimentalResults_2.xlsx'  #Datos obtenidos con mis encuestas
#exp_file = 'ExperimentalResults_3.xlsx'   # Datos de MTURK-771
exp_file = 'ExperimentalResults_4.xlsx'   # Q1 Modificado...

## Initializnig memory and encoding dataset in HD space

In [34]:
%run EncodingDataset.ipynb
Init_mem()

End of memory initialization
Begining to encode dataset...
End of encoding


## Experiment 1

Given a concept it retrieves the "num_concepts" most similar concepts to it (for each different method). A comparison between methods is done by counting the number of coincidences between lists of concepts.

In [42]:
import random 

# List of all concepts in dataset
Concepts = get_concepts_list() 

# Test concepts
Test_Concepts =  ['spoon', 'airplane', 'chair']
sample_size = 100
#Test_Concepts = random.sample(Concepts, sample_size)
print Test_Concepts

num_concepts = 11
coincidence = []

for test_concept in Test_Concepts:
    print "\n\nConcept: ", test_concept
    # Asking closest concept of another concept's definition...
    HDC_sim = HDvector.getLabelSP(Dict[test_concept].getPointer())[:num_concepts]
    HDC_sim = NormalizeHammDist(HDC_sim)
    print "HDC_sim: ", HDC_sim
    
    DatSet_sim = ClosestConcepts(test_concept, num_concepts)
    print "DatSet_sim: ", DatSet_sim

    concept = wn.synset( get_synset(test_concept) )
    # Path similarity
    LC_sim = apply_sim_metric(wn.lch_similarity, num_concepts, concept ) 
    print "LC_sim: ", LC_sim
    WUP_sim = apply_sim_metric(wn.wup_similarity, num_concepts, concept )
    print "WUP_sim: ", WUP_sim
    
    # Information Content
    Res_sim = apply_sim_metric(wn.res_similarity, num_concepts, concept, brown_ic)
    print "Res_sim: ", Res_sim
    JC_sim = apply_sim_metric(wn.jcn_similarity, num_concepts, concept, brown_ic)
    print "JC_sim: ", JC_sim
    
    # Comparing lists agains HDC_sim for counting coincidence.
    # Creating name sets
    HDC_names = set([x[0] for x in HDC_sim])
    DatSet_names = set([x[0] for x in DatSet_sim])
    LC_names = set([x[0] for x in LC_sim])
    WUP_names = set([x[0] for x in WUP_sim])
    Res_names = set([x[0] for x in Res_sim])
    JC_names = set([x[0] for x in JC_sim])
    
    coincidence = coincidence + [[len(HDC_names.intersection(DatSet_names)), 
                                 len(HDC_names.intersection(LC_names)), 
                                 len(HDC_names.intersection(WUP_names)),    
                                 len(HDC_names.intersection(Res_names)), 
                                 len(HDC_names.intersection(JC_names))]]
    print coincidence

coincidence = np.array(coincidence)
# Measuring average against each method.
print "Average: ", np.mean(coincidence, axis = 0)

['spoon', 'airplane', 'chair']


Concept:  spoon
HDC_sim:  [['spoon', 1.0], ['ladle', 0.662], ['spatula', 0.625], ['fork', 0.616], ['colander', 0.61], ['tongs', 0.609], ['pot', 0.588], ['bucket', 0.584], ['bowl', 0.582], ['grater', 0.582], ['peg', 0.582]]
DatSet_sim:  [['spoon', 1.0], ['fork', 0.546], ['ladle', 0.411], ['spatula', 0.395], ['tongs', 0.35], ['colander', 0.336], ['bowl', 0.292], ['strainer', 0.282], ['dish', 0.282], ['peg', 0.28], ['grater', 0.259]]
LC_sim:  [['spoon', 3.638], ['bag', 2.539], ['basket', 2.539], ['bin_(waste)', 2.539], ['box', 2.539], ['cup', 2.539], ['dish', 2.539], ['envelope', 2.539], ['fork', 2.539], ['thimble', 2.539], ['ashtray', 2.251]]
WUP_sim:  [['fork', 0.9], ['spoon', 0.9], ['bag', 0.875], ['basket', 0.875], ['bin_(waste)', 0.875], ['box', 0.875], ['envelope', 0.875], ['thimble', 0.875], ['ashtray', 0.824], ['barrel', 0.824], ['bathtub', 0.824]]
Res_sim:  [['spoon', 11.208], ['fork', 10.574], ['cup', 7.49], ['dish', 7.49], ['plate', 7.49], ['sau

In [36]:
A = np.array([[5, 1, 1, 2, 1], [5, 2, 2, 2, 2], [8, 7, 7, 7, 4], [5, 2, 2, 2, 2], [8, 6, 6, 6, 8], [8, 5, 6, 6, 4], [7, 5, 5, 4, 3], [1, 1, 1, 1, 1], [5, 1, 1, 1, 1], [7, 5, 5, 5, 4], [6, 3, 3, 2, 2], [5, 2, 2, 1, 2], [5, 1, 1, 3, 2], [6, 2, 2, 2, 2], [6, 5, 4, 4, 4], [4, 1, 1, 1, 1], [9, 3, 3, 2, 5], [2, 1, 1, 1, 1], [8, 6, 6, 6, 3], [8, 2, 2, 2, 2], [7, 7, 6, 6, 7], [7, 6, 7, 7, 5], [5, 1, 1, 1, 1], [1, 1, 1, 1, 1]])
print "Average: ", np.mean(A, axis = 0)

Average:  [5.75       3.16666667 3.16666667 3.125      2.83333333]
